<a href="https://colab.research.google.com/github/MikeHankinson/Amazon_Vine_Analysis/blob/main/16_9_1_pySpark_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install PySpark** 
PySPark does not come native to Google Colab

---





In [49]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:15 http://ppa.lau

Use Spark to write directly to the Postgres database (my_data_class_db).   

---



In [50]:
# Download Postgres Driver (1-time operation)
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-02-07 03:33:30--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-02-07 03:33:30 (6.63 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [52]:
# Start a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()


**Extract**: Get Raw Data Stored in AWS S3 into a PySpark Dataframe

---
1. **Import SparkFiles from PySpark** into this notebook. This allows Spark to add a file to the Spark project. 
2. **Read file** in with the read method and combined with the csv() method, which pulls in the CSV stored in SparkFiles and infers the schema. SparkFiles.get() will have Spark retrieve the specified file.
3.  An action is called to **show the first 10 runs** and confirm our data extraction.
4. Complete process to load both **user_data.csv** and 	**user_payment.csv**.


In [53]:
# user_data.csv: Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://hankmikebootcamp.s3.amazonaws.com/user_data.csv"
spark.sparkContext.addFile(url)
user_data_df = spark.read.csv(SparkFiles.get("user_data.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
user_data_df.show(10)


+---+----------+---------+-----------+--------------------+--------------------+-----------+
| id|first_name|last_name|active_user|      street_address|               state|   username|
+---+----------+---------+-----------+--------------------+--------------------+-----------+
|  1|    Cletus|  Lithcow|      false| 78309 Riverside Way|            Virginia|  ibearham0|
|  2|       Caz|   Felgat|      false| 83 Hazelcrest Place|             Alabama|   wwaller1|
|  3|     Kerri|  Crowson|      false|      112 Eliot Pass|      North Carolina|  ichesnut2|
|  4|   Freddie|    Caghy|      false|     15 Merchant Way|            New York|    tsnarr3|
|  5|   Sadella|    Deuss|      false|    079 Acker Avenue|           Tennessee|  fwherrit4|
|  6|    Fraser|  Korneev|       true|  76084 Novick Court|           Minnesota| fstappard5|
|  7|    Demott|   Rapson|       true|    86320 Dahle Park|District of Columbia| lhambling6|
|  8|    Robert|    Poile|      false|1540 Manitowish Hill|           

In [54]:
# user_data.csv: Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://hankmikebootcamp.s3.amazonaws.com/user_payment.csv"
spark.sparkContext.addFile(url)
user_payment_df = spark.read.csv(SparkFiles.get("user_payment.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
user_payment_df.show(10)


+----------+-----------+--------------------+
|billing_id|   username|        cc_encrypted|
+----------+-----------+--------------------+
|         1|  ibearham0|a799fcafe47d7fb19...|
|         2|   wwaller1|a799fcafe47d7fb19...|
|         3|  ichesnut2|a799fcafe47d7fb19...|
|         4|    tsnarr3|a799fcafe47d7fb19...|
|         5|  fwherrit4|a799fcafe47d7fb19...|
|         6| fstappard5|a799fcafe47d7fb19...|
|         7| lhambling6|a799fcafe47d7fb19...|
|         8|     drude7|a799fcafe47d7fb19...|
|         9|  bspawton8|a799fcafe47d7fb19...|
|        10|rmackeller9|a799fcafe47d7fb19...|
+----------+-----------+--------------------+
only showing top 10 rows



**Transform**: Manipulate Data in **user_data_df ** and **user_payment_df**

---

1. Join the two tables
2. Drop rows with null or "not a number" (NaN) values
3. Filter for active users
4. Select columns to create 3 DFs that match the AWS RDS database.  
*   Create a DataFrame to match the active_user table
*   Create a DataFrame to match the billing_info table
*   Create a DataFrame to match the payment_info table

In [55]:
# 1. Join the two Dataframes
joined_df = user_data_df.join(user_payment_df, on="username", how="inner")
joined_df.show(10)

+-----------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|   username| id|first_name|last_name|active_user|      street_address|               state|billing_id|        cc_encrypted|
+-----------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|  ibearham0|  1|    Cletus|  Lithcow|      false| 78309 Riverside Way|            Virginia|         1|a799fcafe47d7fb19...|
|   wwaller1|  2|       Caz|   Felgat|      false| 83 Hazelcrest Place|             Alabama|         2|a799fcafe47d7fb19...|
|  ichesnut2|  3|     Kerri|  Crowson|      false|      112 Eliot Pass|      North Carolina|         3|a799fcafe47d7fb19...|
|    tsnarr3|  4|   Freddie|    Caghy|      false|     15 Merchant Way|            New York|         4|a799fcafe47d7fb19...|
|  fwherrit4|  5|   Sadella|    Deuss|      false|    079 Acker Avenue|           Tennessee|         5|a799fcafe47d7fb19...|


In [56]:
# 2. Drop null values
dropna_df = joined_df.dropna()
dropna_df.show(10)

+-----------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|   username| id|first_name|last_name|active_user|      street_address|               state|billing_id|        cc_encrypted|
+-----------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|  ibearham0|  1|    Cletus|  Lithcow|      false| 78309 Riverside Way|            Virginia|         1|a799fcafe47d7fb19...|
|   wwaller1|  2|       Caz|   Felgat|      false| 83 Hazelcrest Place|             Alabama|         2|a799fcafe47d7fb19...|
|  ichesnut2|  3|     Kerri|  Crowson|      false|      112 Eliot Pass|      North Carolina|         3|a799fcafe47d7fb19...|
|    tsnarr3|  4|   Freddie|    Caghy|      false|     15 Merchant Way|            New York|         4|a799fcafe47d7fb19...|
|  fwherrit4|  5|   Sadella|    Deuss|      false|    079 Acker Avenue|           Tennessee|         5|a799fcafe47d7fb19...|


In [57]:
# 3. Filter for Active Users

# Load in a sql function to use columns
from pyspark.sql.functions import col

# Filter for only columns withy active users
cleaned_df = dropna_df.filter(col("active_user")==True)
cleaned_df.show(10)


+------------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|    username| id|first_name|last_name|active_user|      street_address|               state|billing_id|        cc_encrypted|
+------------+---+----------+---------+-----------+--------------------+--------------------+----------+--------------------+
|  fstappard5|  6|    Fraser|  Korneev|       true|  76084 Novick Court|           Minnesota|         6|a799fcafe47d7fb19...|
|  lhambling6|  7|    Demott|   Rapson|       true|    86320 Dahle Park|District of Columbia|         7|a799fcafe47d7fb19...|
|   wheinerte| 15|   Sadella|    Jaram|       true|7528 Waxwing Terrace|         Connecticut|        15|a799fcafe47d7fb19...|
|droughsedgeg| 17|    Hewitt|  Trammel|       true|    2455 Corry Alley|      North Carolina|        17|a799fcafe47d7fb19...|
|   ydudeniei| 19|       Ted|  Knowlys|       true|      31 South Drive|                Ohio|        19|a799fcafe47d7f

In [58]:
# 4.a. Create a DataFrame to match the active_user table
clean_user_df = cleaned_df.select(["id", "first_name", "last_name", "username"])
clean_user_df.show(10)

+---+----------+---------+------------+
| id|first_name|last_name|    username|
+---+----------+---------+------------+
|  6|    Fraser|  Korneev|  fstappard5|
|  7|    Demott|   Rapson|  lhambling6|
| 15|   Sadella|    Jaram|   wheinerte|
| 17|    Hewitt|  Trammel|droughsedgeg|
| 19|       Ted|  Knowlys|   ydudeniei|
| 23|  Annmarie|   Lafond|    fmyttonm|
| 28|      Toma|   Sokell|  bfletcherr|
| 30|       Ram|  Lefever|    gturleyt|
| 31|    Raddie|  Heindle|   calyukinu|
| 33|    Wallie|     Caws|ckleinlererw|
+---+----------+---------+------------+
only showing top 10 rows



In [59]:
# 4.b.Create a DataFrame to match the billing_info table
clean_billing_df = cleaned_df.select(["billing_id", "street_address", "state", "username"])
clean_billing_df.show(10)

+----------+--------------------+--------------------+------------+
|billing_id|      street_address|               state|    username|
+----------+--------------------+--------------------+------------+
|         6|  76084 Novick Court|           Minnesota|  fstappard5|
|         7|    86320 Dahle Park|District of Columbia|  lhambling6|
|        15|7528 Waxwing Terrace|         Connecticut|   wheinerte|
|        17|    2455 Corry Alley|      North Carolina|droughsedgeg|
|        19|      31 South Drive|                Ohio|   ydudeniei|
|        23|     35 Oriole Place|             Georgia|    fmyttonm|
|        28|39641 Eggendart Hill|            Maryland|  bfletcherr|
|        30|   9969 Laurel Alley|               Texas|    gturleyt|
|        31|   811 Talmadge Road|                Ohio|   calyukinu|
|        33|   9999 Kenwood Pass|              Oregon|ckleinlererw|
+----------+--------------------+--------------------+------------+
only showing top 10 rows



In [60]:
# 4.c. Create a DataFrame to match the payment_info table
clean_payment_df = cleaned_df.select(["billing_id", "cc_encrypted"])
clean_payment_df.show(10)

+----------+--------------------+
|billing_id|        cc_encrypted|
+----------+--------------------+
|         6|a799fcafe47d7fb19...|
|         7|a799fcafe47d7fb19...|
|        15|a799fcafe47d7fb19...|
|        17|a799fcafe47d7fb19...|
|        19|a799fcafe47d7fb19...|
|        23|a799fcafe47d7fb19...|
|        28|a799fcafe47d7fb19...|
|        30|a799fcafe47d7fb19...|
|        31|a799fcafe47d7fb19...|
|        33|a799fcafe47d7fb19...|
+----------+--------------------+
only showing top 10 rows



**Load**: Move transformed raw data **into the AWS Relational Database**.  PySpark will connect to the RDS database to load the DataFrames into the table.

---

1. Configure AWS RDS settings to allow the connection
2. Write the cleaned DataFrames directly to the AWS database
3. Verify work by running querries in pgAdmin on the AWS database to confirm a successful load of data to AWS RDS.  

In [69]:
# 1. Configure AWS RDS settings to allow the connection
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.crj7eeu0fkul.us-east-1.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"root",
          "password": "postgres",
          "driver":"org.postgresql.Driver"}


#   my_data_class_db    mypostgresdb

In [70]:
# 2.Write the cleaned DataFrames directly to the AWS database

# Write DataFrame to active_user table in RDS
clean_user_df.write.jdbc(url=jdbc_url, table='active_user', mode=mode, properties=config)

# Write dataframe to billing_info table in RDS
clean_billing_df.write.jdbc(url=jdbc_url, table='billing_info', mode=mode, properties=config)

# Write dataframe to payment_info table in RDS
clean_payment_df.write.jdbc(url=jdbc_url, table='payment_info', mode=mode, properties=config)

3. Verify work by running querries **in pgAdmin** on the AWS database to confirm a successful load of data to AWS RDS.